# Notebook para modelagem

## Modelagem

### Imports

In [42]:
import logging
import pickle
import sys

from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np
import os

logging.basicConfig(level="INFO", stream=sys.stdout)
import warnings
warnings.filterwarnings('ignore')

### Pré processamento dos dados

In [3]:
dataset = load_iris(as_frame=True)
df_dataset = data1 = pd.DataFrame(data= np.c_[dataset['data'], dataset['target']],
                     columns= dataset['feature_names'] + ['target'])

df_dataset.rename(columns={
    "sepal length (cm)": "sepal_length",
    "sepal width (cm)": "sepal_width",
    "petal length (cm)": "petal_length",
    "petal width (cm)": "petal_width",
}, inplace=True)
df_dataset.head()



,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [4]:
target = df_dataset[["target"]]
target.head()

,target
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [5]:
features = df_dataset[["sepal_length", "sepal_width", "petal_length", "petal_width"]]
features.head()

,sepal_length,sepal_width,petal_length,petal_width
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [6]:
X_train, X_test, y_train, y_test = train_test_split(
    features, 
    target, 
    stratify=target, 
    test_size=0.2, 
    random_state=14
)

### Definindo o pipeline do modelo

A criação do pipeline tem três etapas:
1. Criação do inputer para lidar com valores vazios. Dessa forma, se uma das features tiver um valor vazio, ele será completado com a média dos outros dois valores;
2. Adição de um StandardScaler para normalização dos dados;
3. O estimador, ou modelo, de fato que fará a predição. Para esse exemplo, foi escolhido uma regressão logistica, dada a simplicidade e agilidade no treinamento do modelo.

In [7]:
pipeline = Pipeline(steps = [
        ('Imputer', SimpleImputer(strategy='mean', keep_empty_features=True)),
        ('normalization', StandardScaler()),
        ('estimator', LogisticRegression())
    ]
)

### Fazendo o treinamento do modelo

De forma a encontrar o melhor modelo possível, foi utilizada a técnica de `GridSearchParam` utilizando os possíveis parametros descritos abaixo. Além disso, foi realizada a validação cruzada utilizando cinco folds e a métrica alvo foi definida como AUC.

In [43]:
parameters = {
        'estimator__solver': ['newton-cg'],
        'estimator__tol': [ 0.0001, 0.003, 0.01],
        'estimator__penalty': [None, 'l2'],
    }

model = GridSearchCV(estimator=pipeline,
                            param_grid=parameters,
                            scoring= {"AUC": "roc_auc_ovr"},
                            refit="AUC",
                            cv=5,
                            verbose=1,
                            error_score='raise')

model = model.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits


### Avaliação do modelo

Primeiro fazemos a predição do dataset de teste utilizando o modelo previamente treinado.

In [9]:
y_pred = model.predict(X_test)

Uma vez tendo realizado as predições no dataset de teste, podemos calcular métricas de performance de modelo, como acurácia.

In [10]:
test_acc_score = accuracy_score(y_test, y_pred)
logging.info(f"Accuracy test score: {test_acc_score}")

INFO:root:Accuracy test score: 0.9666666666666667


Uma vez estando satisfeito com o resultado do modelo, podemos persistir esse modelo como um arquivo pickle, para posteriormente só carregarmos esse modelo e usarmos

In [11]:
current_path = os.path.abspath('')
save_model_path = f"{current_path}/../models/logistic_regression_iris.pkl"

with open(save_model_path, "wb") as pickle_file:
    pickle.dump(model, pickle_file)

logging.info(f"Model saved in {save_model_path}")

INFO:root:Model saved in /mnt/3c2f822b-db13-4837-ba6e-3d7b256042cc/repositorios/poc-onnx/modeling/../models/logistic_regression_iris.pkl


### Convertendo para OONX

Import das libs para conversão de scikit-learn para ONNX

In [12]:
from skl2onnx import convert_sklearn
from skl2onnx.common.data_types import FloatTensorType

Carregando o modelo previamente treinado através do arquivo pickle salvo.

In [13]:
with open(f"{current_path}/../models/logistic_regression_iris.pkl", 'rb') as file:
    model = pickle.load(file)

print(model)

GridSearchCV(cv=5, error_score='raise',
             estimator=Pipeline(steps=[('Imputer',
                                        SimpleImputer(keep_empty_features=True)),
                                       ('normalization', StandardScaler()),
                                       ('estimator', LogisticRegression())]),
             param_grid={'estimator__penalty': [None, 'l2'],
                         'estimator__solver': ['newton-cg'],
                         'estimator__tol': [0.0001, 0.003, 0.01]},
             refit='AUC', scoring={'AUC': 'roc_auc_ovr'}, verbose=1)


Observe que ele é do tipo `GridSearchCV`. Dessa forma, devemos utilizar para conversão o melhor estimador gerado pelo `GridSearchCV`. Podemos ter acesso a ele através do atributo `best_estimator_`.

Além disso, também devemos definir o tipo de input que nosso modelo ONNX irá receber, seguindo o mesmo formato utilizado na definição do nosso modelo durante o treinamento. No nosso caso, temos um modelo que recebe 4 inputs no formato float (sepal_length, sepal_width, petal_length, petal_width).

In [14]:
initial_type = [('float_input', FloatTensorType([None, 4]))]
onnx_model = convert_sklearn(model.best_estimator_, initial_types=initial_type)

Uma vez com nosso modelo convertido, podemos salvá-lo, igual fizemos com o modelo no formato pickle. Porém, dessa vez, iremos salvá-lo no formato onnx

In [15]:
with open(f"{current_path}/../models/logistic_regression_iris.onnx", "wb") as f:
    f.write(onnx_model.SerializeToString())

### Testando o modelo ONNX

Import das libs para carregar um modelo ONNX

In [16]:
import onnxruntime as rt

Através do onnx runtime, podemos fazer o carregamento do nosso modelo salvo previamente e garantir que carregamos o modelo certo ao verificar o nome do input que demos na etapa de salvamento.

In [17]:
sess = rt.InferenceSession(f"{current_path}/../models/logistic_regression_iris.onnx")
input_name = sess.get_inputs()[0].name
print("Input Name:", input_name)

Input Name: float_input


Uma vez que modelos ONNX, em ambiente Python, trabalham com dados no formato numpy, iremos converter o dataset de teste para numpy

In [18]:
X_test_np = X_test.astype(np.float32).to_numpy()

Uma vez com os dados prontos para serem utilizados, podemos utilizar o nosso objetivo `InferenceSession` para realizar a inferencia utilizando o input que criamos para o modelo e os dados convertidos para numpy

In [19]:
predictions = sess.run(None, {input_name: X_test_np})

Repare que a predição nos retornou tanto a classe prevista quanto a distribuição de probabilidade para cada classe.

In [20]:
print("Predictions:", predictions)

Predictions: [array([2, 1, 2, 0, 1, 0, 0, 2, 1, 1, 0, 1, 2, 1, 0, 0, 1, 1, 1, 1, 2, 2,
       1, 2, 0, 2, 2, 0, 0, 0], dtype=int64), [{0: 4.3682564864866436e-05, 1: 0.18593932688236237, 2: 0.8140169978141785}, {0: 0.00045449292520061135, 1: 0.9904571175575256, 2: 0.00908839050680399}, {0: 1.0313229437119986e-11, 1: 0.0020974131766706705, 2: 0.9979026317596436}, {0: 0.9978068470954895, 1: 0.0021931955125182867, 2: 3.0318691893009564e-13}, {0: 0.0001320976298302412, 1: 0.990864098072052, 2: 0.00900376308709383}, {0: 0.997782289981842, 1: 0.002217723987996578, 2: 1.130992977223244e-12}, {0: 0.9995378255844116, 1: 0.0004621815460268408, 2: 4.608343871434495e-13}, {0: 8.596723333198497e-09, 1: 0.0024428837932646275, 2: 0.9975571036338806}, {0: 0.0006489232182502747, 1: 0.9930737018585205, 2: 0.0062774126417934895}, {0: 0.0017745760269463062, 1: 0.9635270237922668, 2: 0.03469844162464142}, {0: 0.9987296462059021, 1: 0.0012703877873718739, 2: 2.3457171535040677e-12}, {0: 1.2049142242176458e-0

Podemos conferir se o output do modelo convertido é igual ao modelo antes da conversão de forma a garantir que os resultados são iguais.

In [44]:
pred = model.predict(X_test_np)
print(pred)

[2. 1. 2. 0. 1. 0. 0. 2. 1. 1. 0. 1. 2. 1. 0. 0. 1. 1. 1. 1. 2. 2. 1. 2.
 0. 2. 2. 0. 0. 0.]
